In [100]:
import pandas as pd
import heartpy as hp
import neurokit2 as nk
import numpy as np
import matplotlib.pyplot as plt


In [101]:
# data = pd.read_pickle(f'/home/ilya/Documents/stress-level-by-PPG/working with WESAD/filt_dataset.pkl')
# data = pd.read_pickle(f'/home/ilya/Documents/stress-level-by-PPG/working with WESAD/filt_dataset.pkl')

ppg = np.load('/home/ilya/Documents/stress-level-by-PPG/working with braslet/ppg.npy')
ppg

array([-77.04676535, -71.09153326, -65.49605614, ..., 159.82573384,
       141.34695932, 123.78175725], shape=(23040,))

In [102]:
def drow_data(ppg):
    if len(ppg) > 0:
        wigth = len(ppg) // 256
        figr = plt.figure(figsize=(wigth, 6))
        plt.plot(ppg)
        plt.grid(True)  
        plt.savefig(f'/home/ilya/Documents/stress-level-by-PPG/working with WESAD/penis_blyat.png', dpi=300, bbox_inches='tight')
        plt.close(figr)

In [103]:
drow_data(ppg)

In [104]:
# test_ppg = data['S15']['PPG_medit'][0:20 * 64]
# signals, info = nk.ppg_process(test_ppg, sampling_rate=64)
# nk.ppg_plot(signals, info)

In [105]:
fr_ppg = 256
wind_size = 10 * fr_ppg


dataset = []
feature = ['HRV_MeanNN', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_SDSD', 'HRV_CVNN', 'HRV_CVSD', 'HRV_MedianNN', 'HRV_MadNN', 'HRV_pNN50', 'HRV_pNN20', 'HRV_MinNN', 'HRV_MaxNN', 'HRV_HTI', 'HRV_TINN', 'HRV_HF', 'stress_index']

In [106]:
def get_feature_hp(signal, label):
    _, m = hp.process(hp.scale_data(signal), sample_rate=fr_ppg, clean_rr=True)
    m['stress_index'] = label
    return m

def get_feature_nk(signal, label):
    peaks, _ = nk.ppg_peaks(signal, sampling_rate=fr_ppg, show=False)
    custom_scales = [4, 8, 16]

    hrv_indixes = nk.hrv(
        peaks, 
        sampling_rate=fr_ppg,
        nonlinear_kwargs={"scale": custom_scales}
    )
    hrv_indixes['stress_index'] = label

    filt_hrv_indixes = hrv_indixes[feature]
    return filt_hrv_indixes.iloc[0].to_dict()


In [107]:
def add_to_dataset(ppg, label):
    i = wind_size
    
    while(i + wind_size < len(ppg)):
        slice_ppg = ppg[i - wind_size : i + wind_size]
        dataset.append(get_feature_nk(slice_ppg, label))
        i += wind_size

    # elif len(ppg) > 10 * fr_ppg:
    #     dataset.append(get_feature_nk(ppg, label))

In [108]:
# for person_name, person_data in data.items():
# ppg = person_data['PPG_medit']
add_to_dataset(ppg, 0)

    # ppg = person_data['PPG_stress']
    # add_to_dataset(ppg, person_data['stress_lvl_for_stress'])


/home/ilya/miniconda3/envs/PPG/lib/python3.10/site-packages/neurokit2/hrv/hrv_nonlinear.py:529: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(


ValueError: NeuroKit error: the window cannot contain more data points than the time series. Decrease 'scale'.

In [109]:
pd_dataset = pd.DataFrame(dataset)
pd_dataset.to_csv("ppg_feature.csv", index=False)